In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.cross_decomposition import PLSRegression
from sklearn import metrics
import sklearn.discriminant_analysis as DA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from scipy.stats import shapiro
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from bayes_opt import BayesianOptimization, UtilityFunction
import time
from hyperopt import hp, fmin, tpe
import random, timeit, copy
import matplotlib.pyplot as plt  
import time, sys
from IPython.display import clear_output
import pywt
import pickle
import os


# Helper functions
def remove_outliers(df,columns,n_std):
    """
    This function removes outliers found at +/- n_std's away from the mean. 
    """
    for col in columns:
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] >= mean-(n_std*sd))&(df[col] <= mean+(n_std*sd))]
        
    return df

def latex_tabulate(temp_dFrame):
    for i in range(temp_dFrame.shape[0]):
        temp_mod = temp_dFrame.iloc[i]['Best Model']
        temp_test_acc = temp_dFrame.iloc[i]['Testing Accuracy']
        temp_test_std = temp_dFrame.iloc[i]['Std. Testing Accuracy']
        temp_train_acc = temp_dFrame.iloc[i]['Training Accuracy']
        temp_train_std = temp_dFrame.iloc[i]['Std. Training Accuracy']
        temp_num_pred = temp_dFrame.iloc[i]['# Predictors used in Training']
        print(f'{temp_dFrame.index[i]} & {temp_mod} & {temp_test_acc:.3} ({temp_test_std:.3}) & {temp_train_acc:.3} ({temp_train_std:.3}) & {temp_num_pred} '+r'\\', sep='')
        print('\hline')
    return


def update_progress(progress,frac):
    """
    Helper function to display the progress of the tests.
    Source: https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/
    """
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = str(frac)+" Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

    
# load MIRS.csv dataset and MIRS_features dataset
features = pd.read_csv('MIRS_Data/MIRS_features.csv')
X = dataset = pd.read_csv('Wavelet_Features.csv', header= None)
X.columns = ['slope', 'Spectral Mode', 'LS', 'RS', 'LT', 'RT', 'B', 'pt L', 'pt R', 'max curv', 'k', 'k_central']
dataset.columns = ['slope', 'Spectral Mode', 'LS', 'RS', 'LT', 'RT', 'B', 'pt L', 'pt R', 'max curv', 'k', 'k_central']
MIRS = pd.read_csv("MIRS_Data/MIRS_cleaned.csv")

# Define the Stratified CV split that will be used in grid searches
cv_split = StratifiedKFold(n_splits = 5)
if not os.path.exists("PythonResults"):
    os.mkdir("PythonResults")
    
if not os.path.exists("KDE_Plots"):
    os.mkdir("KDE_Plots")
    
if not os.path.exists("Training_Testing_Accuracy_Plots"):
    os.mkdir("Training_Testing_Accuracy_Plots")
sns.set()

In [8]:
protein = features.loc[:,'Protein_content']
lactose = features.loc[:,'Lactose_content']
urea = features.loc[:,'Urea_Content']
fat = features.loc[:,'Fat_content']
tech_traits = ["RCT", "k20", "a30", "a60", "Casein_micelle_size", "Native_pH", "Heat_stability"]
protein_traits = ['alpha_s1_casein', 'alpha_s2_casein', 'beta_casein', 'kappa_casein','alpha_lactalbumin','beta_lactoglobulin_a', 'beta_lactoglobulin_b']


milk_traits = pd.concat([features.loc[:,"Animal_No_"],protein,lactose,urea,fat], axis = 1)

milk_traits = remove_outliers(milk_traits,["Protein_content", "Lactose_content", "Urea_Content", "Fat_content"],3)

protein_category = pd.qcut(milk_traits.loc[:,'Protein_content'], q = [0,.25,.5,.75,1], labels = ["Q1", "Q2", "Q3", "Q4"])
lactose_category = pd.qcut(milk_traits.loc[:,'Lactose_content'], q = [0,.5,1], labels = ["Q Low", "Q High"])
urea_category = pd.qcut(milk_traits.loc[:,'Urea_Content'], q = [0,.5,1], labels = ["Q Low", "Q High"])
fat_category = pd.qcut(milk_traits.loc[:,'Fat_content'], q = [0,.5,1], labels = ["Q Low", "Q High"])


milk_traits = pd.concat([milk_traits.loc[:,"Animal_No_"],milk_traits.loc[:,"Protein_content"], protein_category, milk_traits.loc[:,"Lactose_content"], lactose_category, milk_traits.loc[:,"Urea_Content"], urea_category, milk_traits.loc[:,"Fat_content"], fat_category], axis = 1 )
milk_traits.columns = ["Animal_No_","Protein", "Protein_Category", "Lactose", "Lactose_Category", "Urea", "Urea_Category", "Fat", "Fat_Category"]
milk_traits.head()

full_data = features.join(MIRS).join(dataset)

full_data = full_data.groupby(["Animal_No_"], sort = False).mean()

full_data = full_data.reset_index()

features_red = full_data.iloc[:,range(0,56)]

MIRS_red = full_data.iloc[:,range(56,1116)]

dataset_red = full_data.iloc[:,range(1116,1128)]

MIRS_no_water = MIRS_red.iloc[:,slice(0,int(np.floor((1600-925)/((5005 - 925)/1060))))].join(
                MIRS_red.iloc[:,slice(int(np.ceil((1710-925)/((5005 - 925)/1060))),int(np.floor((2990-925)/((5005 - 925)/1060))))]).join(
                MIRS_red.iloc[:,slice(int(np.ceil((3690-925)/((5005 - 925)/1060))),int(np.floor((3822-925)/((5005 - 925)/1060))))])
scaler = StandardScaler()
MIRS_no_water_transform = np.log10(1/MIRS_no_water)
MIRS_scaled = pd.DataFrame(scaler.fit_transform(MIRS_no_water_transform))

In [9]:
traits = tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]+protein_traits+["Protein_content"]

describe_table = dict()
for trait in traits:
    describe_table.update([(trait,remove_outliers(pd.DataFrame(features_red.loc[features_red.loc[:,trait] != 0, trait]), [trait],3).describe().values.reshape((8)))])

pd.DataFrame(describe_table, index = ["Count", 'Mean', "SD", "Min", '25%', '50%', '75%', 'Max']).T.round(2)

,Count,Mean,SD,Min,25%,50%,75%,Max
RCT,484.0,20.77,8.79,1.75,14.50,19.62,26.25,47.50
k20,449.0,5.97,3.57,1.25,3.25,5.00,7.75,18.00
a30,421.0,30.59,16.07,0.02,19.16,30.64,42.68,74.90
a60,480.0,31.01,11.29,1.76,23.40,29.24,37.46,66.34
Casein_micelle_size,585.0,186.92,76.21,63.12,156.05,170.80,192.80,968.60
Native_pH,596.0,6.68,0.10,6.34,6.61,6.67,6.75,6.97
Heat_stability,422.0,8.84,6.14,0.58,4.18,6.76,11.43,28.22
Urea_Content,600.0,26.07,15.01,-0.80,14.10,23.85,36.52,66.50
Fat_content,597.0,4.72,1.09,1.62,3.97,4.62,5.37,8.42
Lactose_content,600.0,4.62,0.24,3.90,4.46,4.61,4.78,5.28


In [10]:
wavelet_features = ['slope', 'Spectral Mode', 'LS', 'RS', 'LT', 'RT', 'B', 'pt L', 'pt R', 'max curv','k', 'k_central']
# traits = tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]+protein_traits+["Protein_content"]
pval_cutoff = .05
significant_wavelet_features = dict.fromkeys([trait for trait in traits])

for trait in traits:
    y = features_red.loc[:,[trait]] 
    y = y.dropna()
    y = y[y[trait] != 0]
    y = remove_outliers(y, [trait], 3)[trait]
    sig_feat = []
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
        y_cat = pd.qcut(y,q = [0,.5,1], labels = ['Q Low', 'Q High'])
    if trait in protein_traits+["Protein_content"]:
        y_cat = pd.qcut(y,q = [0,.25,.5,.75,1], labels = ['Q1','Q2','Q3', 'Q4'])

    kde_data = pd.concat([dataset_red.iloc[y_cat.index.tolist()], y_cat], axis= 1)
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
    
        for feat in wavelet_features:
            low = kde_data[y_cat == 'Q Low'][feat]
            high = kde_data[y_cat == 'Q High'][feat]
            if stats.kstest(low, high).pvalue <= pval_cutoff:
                sig_feat.append([feat,stats.kstest(low, high).pvalue])

    if trait in protein_traits+["Protein_content"]:
        for feat in wavelet_features:
            q1 = kde_data[y_cat == 'Q1'][feat]
            q2 = kde_data[y_cat == 'Q2'][feat]
            q3 = kde_data[y_cat == 'Q3'][feat]
            q4 = kde_data[y_cat == 'Q4'][feat]
            pvals = [stats.kstest(q1, q2).pvalue,
                     stats.kstest(q1, q3).pvalue,
                     stats.kstest(q1, q4).pvalue,
                     stats.kstest(q2, q3).pvalue,
                     stats.kstest(q2, q4).pvalue,
                     stats.kstest(q3, q4).pvalue]
            if any(np.array(pvals) <= pval_cutoff):
                sig_feat.append([feat]+[i for i in pvals])
    if len(sig_feat) == 0:
        continue
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
        significant_wavelet_features[trait] = pd.DataFrame(np.array(sig_feat)[:,1], index = np.array(sig_feat)[:,0], columns=['p-value'])
    if trait in protein_traits+["Protein_content"]:
        col_names = ['q1 v q2','q1 v q3','q1 v q4','q2 v q3','q2 v q4','q3 v q4']
        significant_wavelet_features[trait] = pd.DataFrame(np.array(sig_feat)[:,1:7], index = np.array(sig_feat)[:,0], columns= col_names)



In [5]:
wavelet_features = ['slope', 'Spectral Mode', 'LS', 'RS', 'LT', 'RT', 'B', 'pt L', 'pt R', 'max curv','k', 'k_central']
traits = tech_traits+protein_traits+['Lactose_content', 'Urea_Content', 'Fat_content', 'Protein_content']
wavelet_features_ordered_significance = dict.fromkeys([trait for trait in traits])

for trait in traits:
    y = features_red.loc[:,[trait]] 
    y = y.dropna()
    y = y[y[trait] != 0]
    y = remove_outliers(y, [trait], 3)[trait]
    sig_feat = []
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
        y_cat = pd.qcut(y,q = [0,.5,1], labels = ['Q Low', 'Q High'])
    if trait in protein_traits+["Protein_content"]:
        y_cat = pd.qcut(y,q = [0,.25,.5,.75,1], labels = ['Q1','Q2','Q3', 'Q4'])

    kde_data = pd.concat([dataset_red.iloc[y_cat.index.tolist()], y_cat], axis= 1)
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
    
        for feat in wavelet_features:
            low = kde_data[y_cat == 'Q Low'][feat]
            high = kde_data[y_cat == 'Q High'][feat]
            sig_feat.append([feat,float(stats.kstest(low, high).pvalue)])

    if trait in protein_traits+["Protein_content"]:
        for feat in wavelet_features:
            q1 = kde_data[y_cat == 'Q1'][feat]
            q2 = kde_data[y_cat == 'Q2'][feat]
            q3 = kde_data[y_cat == 'Q3'][feat]
            q4 = kde_data[y_cat == 'Q4'][feat]
            pvals = [float(stats.kstest(q1, q2).pvalue),
                     float(stats.kstest(q1, q3).pvalue),
                     float(stats.kstest(q1, q4).pvalue),
                     float(stats.kstest(q2, q3).pvalue),
                     float(stats.kstest(q2, q4).pvalue),
                     float(stats.kstest(q3, q4).pvalue)]
            sig_feat.append([feat]+[p for p in pvals])
            
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
        wavelet_features_ordered_significance[trait] = pd.DataFrame(np.array(sig_feat)[:,1], index = np.array(sig_feat)[:,0], columns=['p-value']).astype('float64')
    if trait in protein_traits+["Protein_content"]:
        col_names = ['q1 v q2','q1 v q3','q1 v q4','q2 v q3','q2 v q4','q3 v q4']
        wavelet_features_ordered_significance[trait] = pd.DataFrame(np.array(sig_feat)[:,1:7], index = np.array(sig_feat)[:,0], columns= col_names).astype('float64')

In [6]:
for trait in traits:
    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
        wavelet_features_ordered_significance[trait] = wavelet_features_ordered_significance[trait].sort_values(by = 'p-value')
    if trait in protein_traits+["Protein_content"]:
        # Sorting Features by average p-value
        wavelet_features_ordered_significance[trait]['mean p-value'] = wavelet_features_ordered_significance[trait].loc[:, col_names].mean(axis = 1)
        wavelet_features_ordered_significance[trait] = wavelet_features_ordered_significance[trait].sort_values(by = 'mean p-value')
        wavelet_features_ordered_significance[trait] = wavelet_features_ordered_significance[trait].drop(columns = ['mean p-value'])

In [7]:
classifiers = ['SVM', 'LDA', 'Logit', 'GNB', 'QDA', 'PLSDA']
# classifiers = ['SVM']
space_dict = dict.fromkeys([classifier for classifier in classifiers])

In [8]:
space_dict['SVM'] = {
                    'C': hp.lognormal('C', 0, 1), # Distribution of tuning parameter space
                    'kernel': hp.choice('kernel',['linear','rbf']),
                    'gamma': hp.uniform('gamma', 0, 1)
            }

space_dict['LDA'] = {
                    'shrinkage': hp.uniform('shrinkage', 0, 1),
            }


space_dict['Logit'] = {
                    'l1_ratio': hp.uniform('l1_ratio', 0, 1),
                    'C': hp.lognormal('C',0, 1)
            }

In [26]:
# from sklearn.metrics import roc_auc_score

# roc_auc_mc_ovr = roc_auc_score(
#     y_test,
#     y_score,
#     multi_class="ovr",
#     average="macro",
# )

multi_class_characteristic = 'accuracy'
binary_class_characteristic = 'roc_auc'

# Changing to area under roc curve did not have a noticable affect on testing accuracy performance

def svc_bo_multi_class(params):
    params = {
        'C': params['C'],
        'kernel': params['kernel'],
        'gamma': params['gamma']
                }
    # Define classification model using parameters
    svc_model = svm.SVC(**params) 
    
    # We want to find parameters that maximize the average cross validation accuracy over the training set
    # Accuracy is minimized so procedure is same for 2 class and 4 class problems
    
    # One vs rest AUC test since we should expect class balance
    best_score = cross_val_score(svc_model, X_train, y_train, scoring = multi_class_characteristic, cv=cv_split, n_jobs = -1).mean()
    return 1 - best_score # Returns 1 - best_score so we have something to minimize

def lda_bo_multi_class(params):
    params = {
        'shrinkage': params['shrinkage'],
            }
    lda_model = DA.LinearDiscriminantAnalysis(shrinkage=params['shrinkage'], solver = 'eigen')
    best_score = cross_val_score(lda_model, X_train, y_train, scoring = multi_class_characteristic, cv=cv_split, n_jobs = -1).mean()

    return 1 - best_score


def logit_cl_bo2_multi_class(params):
    params = {
        'l1_ratio':params['l1_ratio'],
        'C': params['C'],   
            }
    # Elastic Net Penalty is a convex combination of l1 and l2 penalty, no tuning needed for 'penalty' parameter
    logit_bo2 = LogisticRegression(l1_ratio = params['l1_ratio'], C = params['C'], penalty = 'elasticnet', solver = 'saga', max_iter = 15000)
    
    
    best_score = cross_val_score(logit_bo2, X_train, y_train, scoring = multi_class_characteristic, cv=cv_split, n_jobs = -1).mean()

    return 1 - best_score

def svc_bo(params):
    params = {
        'C': params['C'],
        'kernel': params['kernel'],
        'gamma': params['gamma']
                }
    # Define classification model using parameters
    svc_model = svm.SVC(**params) 
    
    # We want to find parameters that maximize the average cross validation accuracy over the training set
    # cv_split = StratifiedKFold(n_splits = 10)
    # Accuracy is minimized so procedure is same for 2 class and 4 class problems
    best_score = cross_val_score(svc_model, X_train, y_train, scoring = binary_class_characteristic, cv=cv_split, n_jobs = -1).mean()
    return 1 - best_score # Returns 1 - best_score so we have something to minimize

def lda_bo(params):
    params = {
        'shrinkage': params['shrinkage'],
            }
    lda_model = DA.LinearDiscriminantAnalysis(shrinkage=params['shrinkage'], solver = 'eigen')
    best_score = cross_val_score(lda_model, X_train, y_train, scoring = binary_class_characteristic, cv= cv_split, n_jobs = -1).mean()
    return 1 - best_score

def logit_cl_bo2(params):
    params = {
        'l1_ratio':params['l1_ratio'],
        'C': params['C'],   
            }
    # Elastic Net Penalty is a convex combination of l1 and l2 penalty, no tuning needed for 'penalty' parameter
    logit_bo2 = LogisticRegression(l1_ratio = params['l1_ratio'], C = params['C'], penalty = 'elasticnet', solver = 'saga', max_iter = 15000)
    best_score = cross_val_score(logit_bo2, X_train, y_train, scoring = binary_class_characteristic, cv=cv_split, n_jobs = -1).mean()
    return 1 - best_score



In [36]:
# Define traits that we are interested in modeling/predicting
traits = tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]+protein_traits+["Protein_content"]
# traits = ["Protein_content","Lactose_content"]

# Define Classificaion Models we're implementing
classifiers = ['SVM', 'LDA', 'Logit', 'GNB', 'QDA', 'PLSDA']
# classifiers = ['SVM']

nreps = 4  # Number of times to repeat procedure for std. and average accuracy scores

# Number of Wavelet Features to include in model in order of significance
num_sig_wavelet_feat = [i for i in range(1,len(wavelet_features) + 1)]

max_bo_evals = 20 # Number of Iterations used in bayesian optimization of tuning parameters

# Create dictionary to store results
test_results_dict = dict.fromkeys([(trait,classifier,acc) for trait in traits for classifier in classifiers for acc in ['Training_Accuracy_mean', 'Training_Accuracy_std', 'Testing_Accuracy_mean', 'Testing_Accuracy_std']], [])
classifier_results_dict = dict.fromkeys([(classifier,trait,num) for classifier in classifiers for trait in traits for num in num_sig_wavelet_feat], [])

classifier_count = 1 # For progress bar

# Iterate procedure over classification models
for classifier in classifiers:
    i = 0 # For progress bar
    
    # Iterate procedure over traits
    for trait in traits:
        j = 0# For progress bar
        update_progress(i/len(traits), str(classifier_count)+'/'+str(len(classifiers)))# For progress bar
        
         # For storing results
        training_accs = [] 
        training_stds = []
        testing_accs = []
        testing_stds = []
        
        # Iterate procedure over Number of Signicant Wavelet Features to implement in order of significance
        for num in num_sig_wavelet_feat:
            k = 0# For progress bar
            
             # For storing results
            temp_training_accs = []
            temp_testing_accs = []
            temp_model_list = []
            
            # Repeat procedure to get a mean and standard deviation
            for count in range(nreps):
                if classifier != 'PLSDA': # PLSDA model has to be performed a little differently
                    
                    y = features_red.loc[:,[trait]] # Define y
                    y = y.dropna() # Drop NA values
                    y = y[y[trait] != 0] # Drop 0 Values
                    y = remove_outliers(y, [trait], 3) # Drop values greater than 3 standard deviations from mean

                    # Consider transforming y data so that it follows a Normal Distribution
                    #index = y.index
                    #y = pd.Series(stats.yeojohnson(y)[0].T[0::-1][0], index = index, name = trait)

                    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
                        y_cat = pd.qcut(y[trait],q = [0,.5,1], labels = ['Q Low', 'Q High'])
                    if trait in protein_traits+["Protein_content"]:
                        y_cat = pd.qcut(y[trait],q = [0,.25,.5,.75,1], labels = ['Q1','Q2','Q3', 'Q4'])

                    # Get the significant wavelet features corresponding to the trait and number of significant features
                    sig_feat = wavelet_features_ordered_significance[trait].index.tolist()[0:num]
                    dataset_red_temp = dataset_red.loc[y_cat.index.tolist()] # Get the appropriate  observations from Wavelet Features
                    dataset_red_temp = dataset_red_temp[sig_feat] # Extract appropriate wavelet features


                    # Define X
                    X = dataset_red_temp
                    
                    
                    # Define a Train-Test split, stratified to the categorical response variable 'y_cat'. 
                    # %20 test_size is used since our lowest obersvation count is 421. 
                    # With 80% data going into training, and a 10-fold cv split, at least 302 observations are used to build ...
                    # ... a model at any given time
                    # New training and testing data is done with each procedure repition 'nreps'
                    X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = .2, stratify = y_cat)
                    
                    # Some Models require/recommend/'occasionally improve with' feature scaling
                    if classifier == 'KNN' or classifier == 'SVM' or classifier == 'Logit': 
                        train_scaler = StandardScaler()
                        test_scaler = StandardScaler()
            
                        X_train = train_scaler.fit_transform(X_train)
                        X_test = test_scaler.fit_transform(X_test)

                    # Retrieve appropriate tuning parameter space
                    space = space_dict[classifier]
                    
                    # Tune classification model and fit
                    if classifier == 'SVM':
                        if trait in protein_traits+["Protein_content"]: #Mult-class bayesian optimizer 
                            best_param = fmin(fn=svc_bo_multi_class,  # Function to minimize
                                            space=space, # Space to minimize over
                                            max_evals=max_bo_evals, # Maximum iterations of function
                                            algo=tpe.suggest, # Run suggested algorithm
                                            verbose = 0) # No intermediate output

                        else:
                            best_param = fmin(fn=svc_bo,  # Function to minimize
                                            space=space, # Space to minimize over
                                            max_evals=max_bo_evals, # Maximum iterations of function
                                            algo=tpe.suggest, # Run suggested algorithm
                                            verbose = 0) # No intermediate output

                        # Define the best parameters as the ones found by the hyperopt function 'fmin'
                        model = svm.SVC(C = best_param['C'], gamma = best_param['gamma'], kernel = ['linear', 'rbf'][best_param['kernel']])
#                         model = svm.SVC(C = best_param['C'])
                        #final training of model
                        model.fit(X_train, y_train) 

                    if classifier == 'LDA':
                        if trait in protein_traits+["Protein_content"]:
                            best_param = fmin(fn=lda_bo_multi_class,
                                            space=space,
                                            max_evals=max_bo_evals,
                                            algo=tpe.suggest,
                                            verbose = 0)
                        else:
                            best_param = fmin(fn=lda_bo,
                                            space=space,
                                            max_evals=max_bo_evals,
                                            algo=tpe.suggest,
                                            verbose = 0)

                        model = DA.LinearDiscriminantAnalysis(shrinkage=best_param['shrinkage'], solver = 'eigen')
                        model.fit(X_train, y_train)

                    if classifier == 'Logit':
                        if trait in protein_traits+["Protein_content"]:
                            logit_best_param = fmin(fn=logit_cl_bo2_multi_class,
                                            space=space,
                                            max_evals=max_bo_evals,
                                            algo=tpe.suggest,
                                            verbose = 0)
                        else:
                            logit_best_param = fmin(fn=logit_cl_bo2,
                                                space=space,
                                                max_evals=max_bo_evals,
                                                algo=tpe.suggest,
                                                verbose = 0)

                        model = LogisticRegression(l1_ratio = logit_best_param['l1_ratio'], C = logit_best_param['C'], penalty = 'elasticnet', solver = 'saga', max_iter = 15000)
                        model.fit(X_train, y_train)

                    if classifier == 'GNB':
                        model = GaussianNB()
                        model.fit(X_train, y_train)

                    if classifier == 'QDA':
                        model = DA.QuadraticDiscriminantAnalysis()
                        model.fit(X_train, y_train)

                    # Store Training and Testing accuracy
                    temp_training_accs.append(accuracy_score(y_train, model.predict(X_train))) 
                    temp_testing_accs.append(accuracy_score(y_test, model.predict(X_test)))
                    temp_model_list.append(model)
                    
                if classifier == 'PLSDA':
                    y = features_red.loc[:,[trait]] 
                    y = y.dropna()
                    y = y[y[trait] != 0]
                    y = remove_outliers(y, [trait], 3)
                    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
                        y_cat = pd.qcut(y[trait],q = [0,.5,1], labels = [0, 1])
                    if trait in protein_traits+["Protein_content"]:
                        y_cat = pd.qcut(y[trait],q = [0,.25,.5,.75,1], labels = ['Q1','Q2','Q3','Q4'])
                        y_cat = pd.get_dummies(y_cat) # Convert 4 level categorical variable into one hot encoded vector
                
                    dataset_red_temp = dataset_red.loc[y_cat.index.tolist()] # Get the appropriate  observations from Wavelet Features
                            
                    # Define X
                    X = dataset_red_temp

                    
                    X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size = .2, stratify = y_cat)
                    # num is now instead number of latent variables in X-scores
                    model = PLSRegression(n_components = num) 
                    model.fit(X_train,y_train)
                    yhat_test = model.predict(X_test)
                    yhat_train = model.predict(X_train)
            
                    if trait in tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]:
                        # Convert predicted probability into categorical variable
                        # .5 is used because other similar predictive models were not able to adjust this. 
                
                        yhat_test_cat = pd.Series((yhat_test > .5).astype('uint8').flatten())
                        yhat_train_cat = pd.Series((yhat_train > .5).astype('uint8').flatten())

                    if trait in protein_traits+["Protein_content"]:
                        # Convert predicted probabilities into categorical variable
                        temp_mat = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
                        yhat_test_cat = [temp_mat[np.argmax(yhat_test[idx])] for idx in range(yhat_test.shape[0])]
                        yhat_train_cat = [temp_mat[np.argmax(yhat_train[idx])] for idx in range(yhat_train.shape[0])]
                    
                    # Store Training and Testing accuracy
                    temp_training_accs.append(accuracy_score(y_train, yhat_train_cat))
                    temp_testing_accs.append(accuracy_score(y_test, yhat_test_cat))
                
                # For progress bar
                k += 1
                update_progress((i + ((j + ((k)/(nreps)))/(len(num_sig_wavelet_feat))))/(len(traits)), str(classifier_count)+'/'+str(len(classifiers)))
            
            # Store the mean and standard deviation of Training Accuracy and Testing Accuracy after...
            # ...'nreps' repitions of procedure
            training_accs.append(np.array(temp_training_accs).mean())
            training_stds.append(np.array(temp_training_accs).std())
            testing_accs.append(np.array(temp_testing_accs).mean())
            testing_stds.append(np.array(temp_testing_accs).std())
            classifier_results_dict[classifier,trait,num] = temp_model_list

            j += 1 # For progress bar
            
        # Store mean and standard deviation of 'nreps' many training & testing accuracies of the given trait and classifier
        test_results_dict[trait,classifier,'Training_Accuracy_mean'] = training_accs
        test_results_dict[trait,classifier,'Training_Accuracy_std'] = training_stds
        test_results_dict[trait,classifier,'Testing_Accuracy_mean'] = testing_accs
        test_results_dict[trait,classifier,'Testing_Accuracy_std'] = testing_stds

        i += 1# For progress bar
    
    classifier_count += 1# For progress bar

6/6 Progress: [##################################################] 100.0%


In [37]:
test_results_dict

{('RCT', 'SVM', 'Training_Accuracy_mean'): [0.6543927648578811,
  0.6905684754521964,
  0.6647286821705426,
  0.6569767441860465,
  0.6608527131782946,
  0.6705426356589147,
  0.6705426356589148,
  0.6770025839793281,
  0.6782945736434108,
  0.6802325581395349,
  0.671188630490956,
  0.6666666666666666],
 ('RCT', 'SVM', 'Training_Accuracy_std'): [0.0038217569658266196,
  0.010070062834702784,
  0.012033550394184201,
  0.009204009592231915,
  0.01160994880853453,
  0.0059206404327594815,
  0.012188606113768203,
  0.010173136788128947,
  0.014444883575580039,
  0.011019846323793242,
  0.008833846467168836,
  0.007751937984496138],
 ('RCT', 'SVM', 'Testing_Accuracy_mean'): [0.7036082474226804,
  0.6185567010309279,
  0.6597938144329897,
  0.672680412371134,
  0.6649484536082474,
  0.6314432989690721,
  0.6494845360824741,
  0.6726804123711341,
  0.6675257731958762,
  0.6288659793814433,
  0.6365979381443299,
  0.6597938144329897],
 ('RCT', 'SVM', 'Testing_Accuracy_std'): [0.03599030939115

In [38]:
over_fitting_detect = .04
under_fitting_detect = 0
test_acc_interval = .015

traits = tech_traits+["Urea_Content", "Fat_content", "Lactose_content"]+protein_traits+["Protein_content"]
# traits = ["Lactose_content", "Protein_content"]

keep_idx_dict= dict.fromkeys([(trait, classifier) for trait in traits for classifier in classifiers])
acceptable_classifiers = []
best_classifiers_dict = dict.fromkeys([trait for trait in traits ])
simplest_classifiers = dict.fromkeys([trait for trait in traits])

temp = test_results_dict
temp = pd.DataFrame(temp, index = [1,2,3,4,5,6,7,8,9,10,11,12])

for trait in traits:
    for classifier in classifiers:

        bool_series = (temp[trait][classifier]['Training_Accuracy_mean'] - \
                                  temp[trait][classifier]['Testing_Accuracy_mean'] > over_fitting_detect) | \
                    (temp[trait][classifier]['Testing_Accuracy_mean'] - temp[trait][classifier]['Training_Accuracy_mean'] > under_fitting_detect)
#         print(temp[trait][classifier])
#         print(bool_series)
        keep_idx = [i for i,val in enumerate(bool_series) if not val]
        keep_idx_dict[trait,classifier] = keep_idx
        

for trait in traits:
    for classifier in classifiers:

        if not len(keep_idx_dict[trait,classifier]) > 0:
            continue # All # of predictors of that classifier and trait were overfitting
        
        new_temp = pd.DataFrame(test_results_dict, index = num_sig_wavelet_feat).T
        temp_dFrame = new_temp.swaplevel(i = 0, j = 1, axis = 0).loc[classifier]
        
 
        keep_classifiers = temp_dFrame.loc[trait].iloc[:,keep_idx_dict[trait,classifier]]

        for i in range(len(keep_idx_dict[trait,classifier])):
            test_acc_mean, test_acc_std = keep_classifiers.loc['Testing_Accuracy_mean'].iloc[i], keep_classifiers.loc['Testing_Accuracy_std'].iloc[i]
            train_acc_mean, train_acc_std = keep_classifiers.loc['Training_Accuracy_mean'].iloc[i], keep_classifiers.loc['Training_Accuracy_std'].iloc[i]
            num_predictor = int(keep_classifiers.iloc[:,i].name)
            acceptable_classifiers.append([trait, classifier, train_acc_mean, train_acc_std, test_acc_mean, test_acc_std, num_predictor])
        

# Simplest Non-over/under fitting models comparable to the best model of that trait and classifier
# Each trait reports mode

# Data frame only containing non-over/under fitting models
acceptable_classifiers = pd.DataFrame(acceptable_classifiers, columns = ['Trait','Classifier','Training Accuracy','Std. Training Accuracy','Testing Accuracy', 'Std. Testing Accuracy', '# Predictors used in Training']).set_index(['Trait','Classifier'])

for trait in traits:
#     print(temp_dFrame2.loc[trait])
    
    # Again find the max testing accuracy now that no classifiers are under/over fitting
    argmax_testing_acc = acceptable_classifiers.loc[trait]['Testing Accuracy'].argmax()
    best_classifiers_dict[trait] = [acceptable_classifiers.loc[trait].iloc[argmax_testing_acc,:].name]+[val for val in acceptable_classifiers.loc[trait].iloc[argmax_testing_acc,:].values[:-1]]+[int(acceptable_classifiers.loc[trait].iloc[argmax_testing_acc,:].values[-1])]
    best_test_acc = acceptable_classifiers.loc[trait].iloc[argmax_testing_acc,:].loc['Testing Accuracy']
    # Find classifiers within test_acc_interval of best test acc
    acceptable_idx = acceptable_classifiers.loc[trait]['Testing Accuracy'].between(best_test_acc - test_acc_interval, best_test_acc + test_acc_interval).values
    temp_dFrame2 = acceptable_classifiers.loc[trait].iloc[acceptable_idx]
    min_predictors = temp_dFrame2['# Predictors used in Training'].min()

    notNone_idx = temp_dFrame2['# Predictors used in Training'].values != None

    min_predictors_idx = (temp_dFrame2.iloc[notNone_idx]['# Predictors used in Training'] == min_predictors).values
    temp_dFrame3 = temp_dFrame2.iloc[notNone_idx].iloc[min_predictors_idx]
    
    # Cannot take argmax again
    argmax_testing_accuracy = [i for i,val in \
                enumerate(temp_dFrame3['Testing Accuracy'] \
                          == temp_dFrame3['Testing Accuracy'].max()) if val]
    if len(argmax_testing_accuracy) < 1:
        print("All (non-over/under fitting) classifiers for trait "+trait+" were not within a threshold of "+str(test_acc_interval)+" of the maximum testing accuracy\n (over_fitting_detect = "+str(over_fitting_detect)+")")
        continue
        
    if len(argmax_testing_accuracy) > 1:
        temp_dFrame4 = temp_dFrame3.iloc[argmax_testing_accuracy,:]# Break ties with minimum std testing accuracy

        argmin_testing_std = [i for i,val in \
                enumerate(temp_dFrame4['Std. Testing Accuracy'] \
                          == temp_dFrame4['Std. Testing Accuracy'].min()) if val]
        simplest_classifiers[trait] = [temp_dFrame4.iloc[argmin_testing_std,:].index[0]]+ [temp_dFrame4.iloc[argmin_testing_std,:].values[0][i] for i in range(4)]+[int(min_predictors)]
        continue
    simplest_classifiers[trait] = [temp_dFrame3.iloc[argmax_testing_accuracy,:].index[0]]+[temp_dFrame3.iloc[argmax_testing_accuracy,:].values[0][i] for i in range(4)]+[int(min_predictors)]

In [39]:
Highest_Testing_Accuracy_Wavelet_Features = pd.DataFrame(best_classifiers_dict, index = ['Model', 'Training Accuracy', 'Std. Training Accuracy', 'Testing Accuracy', 'Std. Testing Accuracy', '# Predictors Used in Training']).T
Simplest_Comparable_Testing_Accuracy_Wavelet_Features = pd.DataFrame(simplest_classifiers, index = ['Model', 'Training Accuracy', 'Std. Training Accuracy', 'Testing Accuracy', 'Std. Testing Accuracy', '# Predictors Used in Training']).T

In [40]:
Highest_Testing_Accuracy_Wavelet_Features

,Model,Training Accuracy,Std. Training Accuracy,Testing Accuracy,Std. Testing Accuracy,# Predictors Used in Training
RCT,PLSDA,0.68863,0.004658,0.680412,0.034192,9
k20,PLSDA,0.697772,0.013997,0.697222,0.036324,8
a30,PLSDA,0.709821,0.011622,0.705882,0.016638,5
a60,SVM,0.679688,0.01289,0.669271,0.025911,12
Casein_micelle_size,GNB,0.560897,0.009375,0.553419,0.036948,3
Native_pH,SVM,0.705357,0.027647,0.689583,0.027243,9
Heat_stability,SVM,0.72997,0.003634,0.729412,0.014409,5
Urea_Content,SVM,0.786979,0.022792,0.770833,0.025345,10
Fat_content,PLSDA,0.653564,0.006179,0.652083,0.020729,11
Lactose_content,Logit,0.742188,0.008119,0.739583,0.02155,7


In [41]:
Simplest_Comparable_Testing_Accuracy_Wavelet_Features

,Model,Training Accuracy,Std. Training Accuracy,Testing Accuracy,Std. Testing Accuracy,# Predictors Used in Training
RCT,PLSDA,0.682817,0.016279,0.667526,0.043966,3
k20,PLSDA,0.699164,0.003412,0.688889,0.013608,4
a30,PLSDA,0.709821,0.011622,0.705882,0.016638,5
a60,SVM,0.677734,0.007452,0.666667,0.04101,2
Casein_micelle_size,Logit,0.548077,0.003205,0.544872,0.041598,2
Native_pH,SVM,0.705357,0.027647,0.689583,0.027243,9
Heat_stability,Logit,0.715875,0.004389,0.714706,0.0174,2
Urea_Content,Logit,0.765104,0.013042,0.758333,0.053684,5
Fat_content,QDA,0.649895,0.007983,0.645833,0.019094,4
Lactose_content,Logit,0.742188,0.008119,0.739583,0.02155,7


In [14]:
sklearn.metrics.get_scorer_names()

NameError: name 'sklearn' is not defined

In [11]:
bool(None)

False

In [12]:
bool('ovr')

True